# Tarea 3

### Marcelino Sánchez Rodríguez
### 12 de marzo de 2023

In [2]:
# Load necessary packages
library(rstan)
library(coda)
library(R2jags)
library(methods)

# set the working directory

setwd("/home/rstudio/tarea2RegAvanz")

#--- Funciones utiles ---
prob<-function(x){
  out<-min(length(x[x>0])/length(x),length(x[x<0])/length(x))
  out
}

source("/home/rstudio/tarea2RegAvanz/claseNietor6.R")

Loading required package: StanHeaders

Loading required package: ggplot2

rstan (Version 2.21.8, GitRev: 2e1f913d3ca3)

For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)


Attaching package: ‘coda’


The following object is masked from ‘package:rstan’:

    traceplot


Loading required package: rjags

Linked to JAGS 4.3.0

Loaded modules: basemod,bugs


Attaching package: ‘R2jags’


The following object is masked from ‘package:coda’:

    traceplot


The following object is masked from ‘package:rstan’:

    traceplot




## Configuramos para comparar los distintos modelos

In [3]:
#-Reading data-
desastres<-read.table("http://gente.itam.mx/lnieto/index_archivos/desastres.txt",header=TRUE)
n<-nrow(desastres)
plot(desastres,type="l")
plot(desastres[2:n,2]-desastres[1:(n-1),2],type="l")

#-Defining data-
data<-list("n"=n,"y"=desastres$No.Desastres,"x"=desastres$Anho)
data2<-list("n"=n,"y"=c(desastres$No.Desastres[1:(n-6)],rep(NA,6)),"x"=desastres$Anho)
data3<-list("n"=n,"y"=desastres$No.Desastres+0.1,"x"=desastres$Anho/1000)

#-Defining inits-
initsa1<-function(){list(beta=rep(0,2),yf1=rep(1,n))}
initsa2<-function(){list(beta=rep(0,2),aux=1,yf1=rep(1,n))}
initsa3<-function(){list(beta=rep(0,2),a=1,yf1=rep(1,n))}
initsb1<-function(){list(beta=rep(0,2),aux2=1,yf1=rep(1,n))}
initsb2<-function(){list(beta=rep(0,2),aux=1,aux2=1,yf1=rep(1,n))}
initsb3<-function(){list(beta=rep(0,2),a=1,aux2=1,yf1=rep(1,n))}
initsc<-function(){list(beta=rep(0,n),tau.b=1,yf1=rep(1,n))}
initsd<-function(){list(mu=rep(1,n),tau.b=1,yf1=rep(1,n))}

#-Selecting parameters to monitor-
parsa1<-c("beta","yf1","mu")
parsa2<-c("beta","yf1","mu","r")
parsa3<-c("beta","yf1","mu","a")
parsb1<-c("beta","yf1","mu","tau")
parsb2<-c("beta","yf1","mu","tau","r")
parsb3<-c("beta","yf1","mu","tau","a")
parscd<-c("tau.b","yf1","mu")


Warning message in file(file, "rt"):
“URL 'http://gente.itam.mx/lnieto/index_archivos/desastres.txt': status was 'Couldn't resolve host name'”


ERROR: Error in file(file, "rt"): cannot open the connection to 'http://gente.itam.mx/lnieto/index_archivos/desastres.txt'


# Modelo 1

In [ ]:
#corremos código

modelCode1 <- 'model
{
#Likelihood
for (i in 1:n) {
#Poisson
	y[i] ~ dpois(mu[i])
	log(mu[i])<-beta[1]+beta[2]*x[i]
	}
#Priors 
for (j in 1:2) { beta[j] ~ dnorm(0,0.001) }

#Prediction 1
#Poisson
for (i in 1:n) { yf1[i] ~ dpois(mu[i]) }

}'

ej6a1.sim <- jagsModel$new(data, initsa1, parsa1, modelCode1,n.thin=2, n.iter=10000, n.burnin=1000, n.chains=2)

ej6a1.sim$runJagsModel()

## Checamos Convergencia

In [ ]:
ej6a1.sim$viewSpace()

ej6a1.sim$graphConvergence()

ej6a1.sim$summaryModel()

ej6a1.sim$dicModelfunc()

## Analizamos modelo